In [1]:
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path2 = os.path.abspath(os.path.join(module_path,'scripts'))
if module_path2 not in sys.path:
    sys.path.append(module_path2)
module_path3 = os.path.abspath(os.path.join(module_path,'data_loader'))
if module_path3 not in sys.path:
    sys.path.append(module_path3)
import single_stock_data
import yahooapi
import datetime

from bokeh.io import output_file, show, output_notebook
from bokeh.layouts import row, column
from bokeh.palettes import Viridis3, d3
from bokeh.plotting import figure
from bokeh.models import CheckboxGroup, CustomJS, Range1d

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#output_file("line_on_off.html", title="line_on_off.py example")
output_notebook()

Loading BokehJS ...

In [2]:
# Return dictionary of variable mapping and callback code (mapping, code)
# Add a way to give checkbox and figure variable names
# Current drawbacks: figure must be named p and checkboxes must be names checkbox
def gen_checkbox_callback(df):
    props = dict(visible=False, line_width=4, line_alpha=0.7)
    template = """
    if (checkbox.active.indexOf(variable_index) > -1) {
        line_name.visible = true
    } else {
        line_name.visible = false
    }"""
    code = """
    console.log("cb_obj")
    console.log(cb_obj)
    console.log("cb_data")
    console.log(cb_data)
    """
    mapping = {}
    labels = []
    i = 0
    color_set = d3['Category10'][10]
    for column in df.columns:
        code += template.replace("line_name","line"+str(i)).replace("variable_index",str(i))
        iprops = props
        iprops["color"] = color_set[i % (len(color_set))]
        #iprops["legend"] = "Line " + str(i)
        exec("l"+str(i)+" =p.line(df.index, df[column], **iprops)")
        exec('mapping[\"line'+str(i)+'\"]=l'+str(i))
        labels.append(column)
        i+=1
    
    return (code, mapping, labels)

In [3]:
#def date_parser_function(dateAsString):
#    return datetime.datetime.strptime(dateAsString,"%m/%d/%Y")

#df = pd.read_csv("~/Programming/Python/BT/tests/TimeSeries",
#                index_col="Date",
#                parse_dates=["Date"],
#                date_parser=date_parser_function,
#                sep="\t")

stock = ["GOOGL", "AAPL", "FB"]
startdate = datetime.datetime(2000,1,1)
enddate = datetime.datetime(2018,3,19)
data_panel = yahooapi.get_data(stock,startdate,enddate)
df = data_panel['Close']

In [4]:
#print(code_fromfunc)
#print(mapping_fromfunc)
p = figure(x_axis_type='datetime')
myRange = Range1d(0,3000)
p.y_range = myRange

code_fromfunc, mapping_fromfunc, labels_fromfunc = gen_checkbox_callback(df)

callback_fromfunc = CustomJS(code=code_fromfunc, args={})
checkbox = CheckboxGroup(labels=labels_fromfunc, callback=callback_fromfunc, width=500)
mapping_fromfunc['checkbox'] = checkbox
callback_fromfunc.args = mapping_fromfunc

layout = row(p, checkbox, sizing_mode='fixed')
show(layout)

AttributeError: 'Series' object has no attribute 'columns'

In [7]:
print(data_panel.loc['GOOGL'])

                Close       High        Low       Open    Volume
Date                                                            
2004-08-19    50.2161    52.0804    48.0265    50.0535  44662908
2004-08-20    54.2075    54.5928    50.2987    50.7892  22778271
2004-08-23    54.7530    56.7950    54.5778    55.4337  18234539
2004-08-24    52.4858    55.8541    51.8352    55.6739  15241412
2004-08-25    53.0514    54.0523    51.9903    52.5308   9130694
2004-08-26    54.0073    54.0273    52.3807    52.5258   7077252
2004-08-27    53.1264    54.3626    52.8962    54.1024   6201059
2004-08-30    51.0544    52.7961    51.0544    52.7961   5191609
2004-08-31    51.2346    51.9052    51.1295    51.9052   4922084
2004-09-01    50.1736    51.5349    49.8833    51.3998   9123695
2004-09-02    50.8042    51.2346    49.5179    49.5930  15066623
2004-09-03    50.0535    50.9193    49.7081    50.5239   5149420
2004-09-06    50.0535    50.9193    49.7081    50.5239         0
2004-09-07    50.7992    